# FAQs

## How to...

### ... get data for another symbol?

In [1]:
from pybroker import ExecContext, Strategy, YFinance, highest

def exec_fn(ctx: ExecContext):
    if ctx.symbol == 'NVDA':
        other_bar_data = ctx.foreign('AMD')
        other_highest = ctx.indicator('high_10d', 'AMD')
        
strategy = Strategy(YFinance(), start_date='1/1/2022', end_date='1/1/2023')
strategy.add_execution(
   exec_fn, ['NVDA', 'AMD'], indicators=highest('high_10d', 'close', period=10))
result = strategy.backtest(calc_bootstrap=False)

Backtesting: 2022-01-01 00:00:00 to 2023-01-01 00:00:00

Loading bar data...
[*********************100%***********************]  2 of 2 completed
Loaded bar data: 0:00:00 

Computing indicators...


100% (2 of 2) |##########################| Elapsed Time: 0:00:00 Time:  0:00:00



Test split: 2022-01-03 00:00:00 to 2022-12-30 00:00:00


100% (251 of 251) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:02


You can also retrieve models, predictions, and other data for other symbols. [For more information, refer to the ExecContext reference documentation.](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext)

### ... set a limit price?

Set [buy_limit_price](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.buy_limit_price) or [sell_limit_price](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.sell_limit_price):

In [2]:
from pybroker import ExecContext, Strategy, YFinance

def buy_fn(ctx: ExecContext):
    if not ctx.long_pos():
        ctx.buy_shares = 100
        ctx.buy_limit_price = ctx.close[-1] * 0.99
        ctx.hold_bars = 10
        
strategy = Strategy(YFinance(), start_date='3/1/2022', end_date='1/1/2023')
strategy.add_execution(buy_fn, 'SPY')
result = strategy.backtest(calc_bootstrap=False)
result.orders.head(10)

Backtesting: 2022-03-01 00:00:00 to 2023-01-01 00:00:00

Loading bar data...
[*********************100%***********************]  1 of 1 completed
Loaded bar data: 0:00:00 

Test split: 2022-03-01 00:00:00 to 2022-12-30 00:00:00


100% (212 of 212) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:00


,type,symbol,date,shares,limit_price,fill_price,fees
id,,,,,,,
1,buy,SPY,2022-03-04,100,431.35,430.62,0.0
2,sell,SPY,2022-03-18,100,NaN,441.04,0.0
3,buy,SPY,2022-04-06,100,446.52,446.20,0.0
4,sell,SPY,2022-04-21,100,NaN,443.56,0.0
5,buy,SPY,2022-04-22,100,433.68,431.76,0.0
6,sell,SPY,2022-05-06,100,NaN,410.26,0.0
7,buy,SPY,2022-05-09,100,407.23,401.46,0.0
8,sell,SPY,2022-05-23,100,NaN,394.06,0.0
9,buy,SPY,2022-05-24,100,392.95,391.05,0.0


### ... get current positions?

In [3]:
from pybroker import ExecContext, Strategy, YFinance

def exec_fn(ctx: ExecContext):
    # Get all positions.
    all_positions = tuple(ctx.positions())
    # Get all long positions.
    long_positions = tuple(ctx.long_positions())
    # Get all short positions.
    short_positions = tuple(ctx.short_positions())
    # Get long position for current ctx.symbol.
    long_position = ctx.long_pos()
    # Get short position for a symbol.
    short_position = ctx.short_pos('QQQ')
        
strategy = Strategy(YFinance(), start_date='3/1/2022', end_date='1/1/2023')
strategy.add_execution(exec_fn, ['SPY', 'QQQ'])
result = strategy.backtest(calc_bootstrap=False)

Backtesting: 2022-03-01 00:00:00 to 2023-01-01 00:00:00

Loading bar data...
[*********************100%***********************]  2 of 2 completed
Loaded bar data: 0:00:00 

Test split: 2022-03-01 00:00:00 to 2022-12-30 00:00:00


100% (212 of 212) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:00


[See the Position class for more information.](https://www.pybroker.com/en/latest/reference/pybroker.portfolio.html#pybroker.portfolio.Position)

### ... place an order more than one bar ahead?

Use the [buy_delay](https://www.pybroker.com/en/latest/reference/pybroker.config.html#pybroker.config.StrategyConfig.buy_delay) and [sell_delay](https://www.pybroker.com/en/latest/reference/pybroker.config.html#pybroker.config.StrategyConfig.sell_delay) configuration options:

In [4]:
from pybroker import ExecContext, Strategy, StrategyConfig, YFinance

def buy_fn(ctx: ExecContext):
    if not tuple(ctx.pending_orders()) and not ctx.long_pos():
        ctx.buy_shares = 100
        ctx.hold_bars = 1

config = StrategyConfig(buy_delay=5)
strategy = Strategy(YFinance(), start_date='3/1/2022', end_date='1/1/2023', config=config)
strategy.add_execution(buy_fn, 'SPY')
result = strategy.backtest(calc_bootstrap=False)
result.orders.head(10)

Backtesting: 2022-03-01 00:00:00 to 2023-01-01 00:00:00

Loading bar data...
[*********************100%***********************]  1 of 1 completed
Loaded bar data: 0:00:00 

Test split: 2022-03-01 00:00:00 to 2022-12-30 00:00:00


100% (212 of 212) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:00


,type,symbol,date,shares,limit_price,fill_price,fees
id,,,,,,,
1,buy,SPY,2022-03-08,100,NaN,421.16,0.0
2,sell,SPY,2022-03-09,100,NaN,426.17,0.0
3,buy,SPY,2022-03-16,100,NaN,430.24,0.0
4,sell,SPY,2022-03-17,100,NaN,437.13,0.0
5,buy,SPY,2022-03-24,100,NaN,447.63,0.0
6,sell,SPY,2022-03-25,100,NaN,450.71,0.0
7,buy,SPY,2022-04-01,100,NaN,451.30,0.0
8,sell,SPY,2022-04-04,100,NaN,454.59,0.0
9,buy,SPY,2022-04-11,100,NaN,442.20,0.0


### ... cancel pending orders?

See the [cancel_pending_order](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.cancel_pending_order) and [cancel_all_pending_orders](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.cancel_all_pending_orders) methods.

In [5]:
from pybroker import ExecContext, Strategy, StrategyConfig, YFinance

def buy_fn(ctx: ExecContext):
    pending = tuple(ctx.pending_orders())
    if not pending and not ctx.long_pos():
        ctx.buy_shares = 100
        ctx.hold_bars = 1
    if pending and ctx.close[-1] < 430:
        ctx.cancel_all_pending_orders(ctx.symbol)

config = StrategyConfig(buy_delay=5)
strategy = Strategy(YFinance(), start_date='3/1/2022', end_date='1/1/2023', config=config)
strategy.add_execution(buy_fn, 'SPY')
result = strategy.backtest(calc_bootstrap=False)
result.orders.head(10)

Backtesting: 2022-03-01 00:00:00 to 2023-01-01 00:00:00

Loading bar data...
[*********************100%***********************]  1 of 1 completed
Loaded bar data: 0:00:00 

Test split: 2022-03-01 00:00:00 to 2022-12-30 00:00:00


100% (212 of 212) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:00


,type,symbol,date,shares,limit_price,fill_price,fees
id,,,,,,,
1,buy,SPY,2022-03-23,100,NaN,446.10,0.0
2,sell,SPY,2022-03-24,100,NaN,447.63,0.0
3,buy,SPY,2022-03-31,100,NaN,454.96,0.0
4,sell,SPY,2022-04-01,100,NaN,451.30,0.0
5,buy,SPY,2022-04-08,100,NaN,448.29,0.0
6,sell,SPY,2022-04-11,100,NaN,442.20,0.0
7,buy,SPY,2022-04-19,100,NaN,441.74,0.0
8,sell,SPY,2022-04-20,100,NaN,445.53,0.0


### ... persist data across bars?

Use the [ExecContext#session](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.session) dictionary:

In [6]:
from pybroker import ExecContext, Strategy, YFinance

def buy_fn(ctx: ExecContext):
    if not ctx.long_pos():
        ctx.buy_shares = 100
        ctx.hold_bars = 1
        count = ctx.session.get('entry_count', 0)
        ctx.session['entry_count'] = count + 1

strategy = Strategy(YFinance(), start_date='1/1/2022', end_date='1/1/2023')
strategy.add_execution(buy_fn, 'SPY')
result = strategy.backtest(calc_bootstrap=False)

Backtesting: 2022-01-01 00:00:00 to 2023-01-01 00:00:00

Loading bar data...
[*********************100%***********************]  1 of 1 completed
Loaded bar data: 0:00:00 

Test split: 2022-01-03 00:00:00 to 2022-12-30 00:00:00


100% (251 of 251) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:00


### ... exit a position?

Use [sell_all_shares()](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.sell_all_shares) or [cover_all_shares()](https://www.pybroker.com/en/latest/reference/pybroker.context.html#pybroker.context.ExecContext.cover_all_shares) to liquidate a position:

In [7]:
from pybroker import ExecContext, Strategy, YFinance

def buy_fn(ctx: ExecContext):
    pos = ctx.long_pos()
    if not pos:
        ctx.buy_shares = 100
    elif pos.bars > 30:
        ctx.sell_all_shares()

strategy = Strategy(YFinance(), start_date='1/1/2022', end_date='1/1/2023')
strategy.add_execution(buy_fn, 'SPY')
result = strategy.backtest(calc_bootstrap=False)
result.trades

Backtesting: 2022-01-01 00:00:00 to 2023-01-01 00:00:00

Loading bar data...
[*********************100%***********************]  1 of 1 completed
Loaded bar data: 0:00:00 

Test split: 2022-01-03 00:00:00 to 2022-12-30 00:00:00


100% (251 of 251) |######################| Elapsed Time: 0:00:00 Time:  0:00:00



Finished backtest: 0:00:00


,type,symbol,entry_date,exit_date,entry,exit,shares,pnl,return_pct,agg_pnl,bars,pnl_per_bar,stop
id,,,,,,,,,,,,,
1,long,SPY,2022-01-04,2022-02-18,477.78,435.24,100,-4254.0,-8.90,-4254.0,32,-132.94,None
2,long,SPY,2022-02-22,2022-04-07,430.68,447.11,100,1643.0,3.81,-2611.0,32,51.34,None
3,long,SPY,2022-04-08,2022-05-25,448.29,395.67,100,-5262.0,-11.74,-7873.0,32,-164.44,None
4,long,SPY,2022-05-26,2022-07-14,402.75,375.04,100,-2771.0,-6.88,-10644.0,32,-86.59,None
5,long,SPY,2022-07-15,2022-08-30,382.90,400.05,100,1715.0,4.48,-8929.0,32,53.59,None
6,long,SPY,2022-08-31,2022-10-17,398.14,362.63,100,-3551.0,-8.92,-12480.0,32,-110.97,None
7,long,SPY,2022-10-18,2022-12-02,371.49,405.00,100,3351.0,9.02,-9129.0,32,104.72,None
